# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
# Checking the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# The connection to a Cassandra instance local machine (127.0.0.1) 

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
#creating a session to establish connection and begin executing queries
    session = cluster.connect()
except Exception as e:
    print(e)


#### Create Keyspace

In [8]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

In [10]:
#Creating tables 
query1 = "CREATE TABLE IF NOT EXISTS song_playlist_item_session"
query1 = query1+ "(sessionId int, itemInSession int, artist text,length float, song text, PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query1)
except Exception as e:
    print (e)
    
query2 = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query2 = query2+ "(sessionId int, userId int, itemInSession int, firstName text, lastName text, artist text,length float, song text, PRIMARY KEY ((sessionId, userId), itemInSession))"

try:
    session.execute(query2)
except Exception as e:
    print (e)
    
query3 = "CREATE TABLE IF NOT EXISTS users_music_app_history"
query3 = query3+ "(song text,artist text,length float, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"

try:
    session.execute(query3)
except Exception as e:
    print (e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [11]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query1= "SELECT artist, song, length FROM song_playlist_item_session WHERE sessionId= 338 AND itemInSession = 4"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)
                    

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [12]:
# Setting up the CSV file to import data from CSV file into the created tables
df = pd.read_csv ("event_datafile_new.csv")

query1 = "INSERT INTO song_playlist_item_session (sessionId, itemInSession, artist,length, song)"
query1 = query1 + "VALUES (%s, %s, %s, %s,%s)"
for index , row in df.iterrows():
    try:
## Assigning the INSERT statements into the `query1` variable

        session.execute(query1,(row.sessionId,row.itemInSession, row.artist, row.length, row.song))
    except Exception as e:
        print(e)
        
query2 = "INSERT INTO song_playlist_session (sessionId, userId, itemInSession, firstName, lastName, artist,length, song)"
query2 = query2 + "VALUES (%s, %s, %s, %s,%s, %s, %s,%s)"
for index , row in df.iterrows():
    try:
## Assigning the INSERT statements into the `query2` variable

        session.execute(query2,(row.sessionId,row.userId, row.itemInSession, row.firstName, row.lastName, row.artist, row.length, row.song))
    except Exception as e:
        print(e)
        
query3 = "INSERT INTO users_music_app_history (song, artist, length, userId, firstName, lastName)"
query3 = query3 + "VALUES (%s, %s, %s, %s,%s, %s)"
for index , row in df.iterrows():
    try:
## Assigning the INSERT statements into the `query3` variable

        session.execute(query3,(row.song, row.artist, row.length, row.userId,row.firstName, row.lastName))
    except Exception as e:
        print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
## Verifying the data was entered into the tables
#query= "SELECT * FROM song_playlist_item_session"
#query= "SELECT * FROM song_playlist_session"
query= "SELECT * FROM users_music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)


Row(song="Wonder What's Next", userid=49, artist='Chevelle', firstname='Chloe', lastname='Cuevas', length=250.0697479248047)
Row(song="In The Dragon's Den", userid=49, artist='Symphony X', firstname='Chloe', lastname='Cuevas', length=238.54974365234375)
Row(song='Too Tough (1994 Digital Remaster)', userid=44, artist='The Rolling Stones', firstname='Aleena', lastname='Kirby', length=231.78404235839844)
Row(song='Rio De Janeiro Blue (Album Version)', userid=49, artist='Randy Crawford', firstname='Chloe', lastname='Cuevas', length=251.40199279785156)
Row(song='My Place', userid=15, artist='Nelly / Jaheim', firstname='Lily', lastname='Koch', length=336.5611877441406)
Row(song='My Place', userid=73, artist='Nelly / Jaheim', firstname='Jacob', lastname='Klein', length=336.5611877441406)
Row(song='The Lucky Ones', userid=24, artist='Pride Tiger', firstname='Layla', lastname='Griffin', length=215.77098083496094)
Row(song='I Want You Now', userid=80, artist='The Feeling', firstname='Tegan', las

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [15]:
## Retrieving the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query1= "SELECT artist, song, length FROM song_playlist_item_session WHERE sessionId= 338 AND itemInSession = 4"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)
                    

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [17]:
## Retrieving the name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2= "SELECT artist, song, itemInSession, firstName, lastName FROM song_playlist_session WHERE userId= 10 AND sessionId = 182 ORDER BY itemInSession DESC"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)                

Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', iteminsession=3, firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', iteminsession=2, firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', iteminsession=1, firstname='Sylvie', lastname='Cruz')
Row(artist='Down To The Bone', song="Keep On Keepin' On", iteminsession=0, firstname='Sylvie', lastname='Cruz')


In [18]:
##Retrieving every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query3= "SELECT firstName, lastName FROM users_music_app_history WHERE song ='All Hands Against His Own'"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)#.firstName, row.lastName)
                    

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [19]:
## Dropping song_playlist_item_session table before closing out the sessions
query= "drop table song_playlist_item_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
    

In [26]:
## Dropping song_playlist_session table before closing out the sessions
query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table song_playlist_session"


In [27]:
 ## Dropping users_music_app_history table before closing out the sessions

query = "drop table users_music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()